In [1]:
import numpy as np
import geopandas as gpd
import ultralytics
import os
from ultralytics import YOLO
from IPython.display import display, Image
from roboflow import Roboflow
from shapely.geometry import Polygon

ultralytics.checks()

predict_dir = 'predict'

Ultralytics YOLOv8.0.196  Python-3.11.8 torch-2.3.1+cpu CPU (Intel Core(TM) i7-9700 3.00GHz)
Setup complete  (8 CPUs, 31.8 GB RAM, 572.1/943.7 GB disk)


In [2]:
# should i download teh dataset and use that instead?
rf = Roboflow(api_key="JQ9e5GaVaHhNHRURLub7")
project = rf.workspace("fip").project("fire-insurance")
version = project.version(14)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 43.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to Fire-Insurance-14 in yolov8:: 100%|██████████████| 312/312 [00:00<00:00, 2424.79it/s]


In [3]:
dataset.location

'C:\\Users\\whiteco7\\Notebooks\\fip_dl_may2024\\Fire-Insurance-14'

In [4]:
# train the model
n_epochs = 300
bs = -1
verbose = True
rng = 0
validate = True
imgsz = 640

model = YOLO('yolov8x-seg.pt') # large version of model

results = model.train(
    data=dataset.location + '/data.yaml',
    epochs=n_epochs,
    batch=bs,
    verbose=verbose,
    seed=rng,
    val=validate,
    imgsz=imgsz
)

# !yolo task=segment mode=train model=yolov8s-seg.pt data={dataset.location}/data.yaml epochs=300 imgsz=640

New https://pypi.org/project/ultralytics/8.2.89 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.11.8 torch-2.3.1+cpu CPU (Intel Core(TM) i7-9700 3.00GHz)
engine\trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=C:\Users\whiteco7\Notebooks\fip_dl_may2024\Fire-Insurance-14/data.yaml, epochs=300, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

KeyboardInterrupt: 

In [5]:
model = YOLO(f'runs/segment/train/weights/best.pt')

In [6]:
# Apply geographic transformation from world file to model prediction results
def georef(x, y, world_data):
    xscale, yskew, xskew, yscale, xpos, ypos = world_data

    X_proj = xpos + (xscale * x) + (xskew * y)
    Y_proj = ypos + (yscale * y) + (yskew * x)

    return np.array([X_proj, Y_proj])

In [7]:
def predict_geo(file, world_file):
    results = model.predict(source=file, conf=0.25)
    
    # Create an empty list to store the geometries and class labels
    geometries = []
    class_labels = []
    classNames = ['building-footprints']
    
    new_geoms = []
    
    for r in results:
        if r.masks:
            for item in r.masks.xy:
                new_coords = []
                for coords in item:
                    c = georef(coords[0], coords[1], world_data)
                    new_coords.append(c)
                new_geoms.append(Polygon(new_coords))
        else:
            print('No features detected in {}'.format(file))
    
    # Create a GeoDataFrame from the list of geometries and class labels
    gdf = gpd.GeoDataFrame({'geometry': new_geoms}) # also create field for identified class type

    if len(gdf) > 0:    
        # save to shapefile
        output_shapefile = os.path.splitext(file)[0] + '.shp'
        gdf.to_file(output_shapefile, driver='ESRI Shapefile',  crs='EPSG:32617', engine='fiona')

In [8]:
# Run model on all images in 'predict' folder.
# Assign spatial info (utm zone 17) using each image's respective world file
for root, _, files in os.walk(predict_dir):
    for file in files:
        if file.lower().endswith(('.tif', '.jpg')):
            image_file = os.path.join(root, file)
            world_file = None
            
            if file.lower().endswith('.tif'):
                world_file = os.path.splitext(image_file)[0] + '.tfw'
            elif file.lower().endswith('.jpg'):
                world_file = os.path.splitext(image_file)[0] + '.jgw'
            
            if world_file and os.path.exists(world_file):

                with open (world_file, 'r') as wf:
                    world_data = []
                    for line in wf:
                        world_data.append(float(line.strip()))

                predict_geo(image_file, world_data)
                
            else:
                print(f"No corresponding world file found for {image_file}")

No corresponding world file found for predict\912.7135g57.2br oos_11.jpg



image 1/1 C:\Users\whiteco7\Notebooks\fip_dl_may2024\predict\912.7135g57.2br oos_11a.tif: 640x544 52 building-footprintss, 753.0ms
Speed: 13.0ms preprocess, 753.0ms inference, 40.9ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 C:\Users\whiteco7\Notebooks\fip_dl_may2024\predict\912.7135g57.2br oos_22.jpg: 640x544 142 building-footprintss, 657.2ms
Speed: 4.0ms preprocess, 657.2ms inference, 81.8ms postprocess per image at shape (1, 3, 640, 544)
